In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv("Processed_Train.csv")

# Data preprocessing
# Drop any columns that are not relevant for prediction
data = data.drop(["ID"], axis=1)

# Encode categorical variables (you may need to use more advanced encoding techniques)
#data = pd.get_dummies(data, columns=["LandPreparationMethod", "CropEstMethod", "TransDetFactor", "Stubble_use"])

# Split data into features (X) and target (y)
X = data.drop(["Yield"], axis=1)
y = data["Yield"]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape data for LSTM input (assuming a time sequence)
# You may need to adjust the sequence length based on your data
sequence_length = 10  # Example sequence length
X_train = np.reshape(X_train, (X_train.shape[0], sequence_length, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], sequence_length, X_test.shape[1]))

# Define the LSTM model
model = keras.Sequential([
    keras.layers.LSTM(64, activation='relu', input_shape=(sequence_length, X_train.shape[2]), return_sequences=True),
    keras.layers.LSTM(64, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=2)

# Evaluate the model
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Visualize training history
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

